In [9]:
import os
import wave
import contextlib
import pandas as pd
import shutil
import audio_metadata

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [4]:
print(os.getcwd())

/Users/mushroomgoo/Documents/S1course/API/ScreamClassifier/preprocessing


# Pre-Processing the files
Creating a band name-song name-video id lookup and renaming the file to the video id
Video ID will be used to represent the files and the annotations, can be mapped back to song name using the lookup as required

In [5]:
df=pd.DataFrame()
band_name=[]
album_name=[]
video_id=[]
release_year=[]
song_name=[]
bit_depth=[] 
bitrate=[]
channels=[]
duration=[]
sample_rate=[]


loc='./resources/dataset/YTDownloads/'
for file_name in os.listdir(loc):
    if file_name.endswith(".wav"):
        #print(file_name)
        m1=audio_metadata.load(loc+file_name)
        bit_depth.append(m1['streaminfo']['bit_depth'])
        bitrate.append(m1['streaminfo']['bitrate'])
        channels.append(m1['streaminfo']['channels'])
        duration.append(m1['streaminfo']['duration'])
        sample_rate.append(m1['streaminfo']['sample_rate'])

        name=file_name[:-4]
        #print(name)
        #print(name[-11:])
        band_name_split=name.split(" - ")
        song_name_split=band_name_split[1].split(" (")
        if len(song_name_split) == 1:
            song_name_split=band_name_split[1].split("-")
        #print(band_name_split[0])
        #print(song_name_split[0])
        band=band_name_split[0]
        band_name.append(band)
        song=song_name_split[0]
        song_name.append(song)
        vid_id=name[-11:]
        video_id.append(vid_id)

        album_name.append("")
        release_year.append("")
        src='./resources/dataset/'+file_name
        dst='./resources/dataset/final/'+vid_id+'.wav'
        #shutil.copy2(src,dst)
        
    else:
        continue

df['band_name']=band_name
df['album_name']=album_name
df['song_name']=song_name
df['release_year']=release_year
df['duration_seconds']=duration
df['video_id']=video_id
df['bit_depth']=bit_depth
df['bitrate']=bitrate
df['channels']=channels

df['sample_rate']=sample_rate
df

NameError: name 'pd' is not defined

In [60]:
df.to_csv('./resources/dataset/lookup_post120.csv',header=True, index=False,encoding='utf-8-sig')

In [54]:
audio_loc='./resources/dataset/Audio/'
annotation_loc='./resources/dataset/Annotations'
lookup_loc='./resources/dataset/lookup.csv'

In [55]:
"""
lut=pd.read_csv(lookup_loc)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(lut)"""

"\nwith pd.option_context('display.max_rows', None, 'display.max_columns', None):\n    display(lut)"

In [50]:
import audio_metadata
i=1
loc='./resources/dataset/YTDownloads/'
for file_name in os.listdir(audio_loc):
    if file_name.endswith(".wav"):
        #print(audio_loc+file_name)
        m1=audio_metadata.load(audio_loc+file_name)

In [51]:
m1['streaminfo']['bit_depth']
m1['streaminfo']['bitrate']
m1['streaminfo']['channels']
m1['streaminfo']['duration']
m1['streaminfo']['sample_rate']

44100

In [56]:
m1['streaminfo']['duration']

337.36272108843536

# RUN THE BELOW TWO PARAGRAPHS EVERY TIME THE DATASET IS UPDATED/ADDED TO

# Processing the annotatated values into the easy to work with schema

In [ ]:
loc='./resources/dataset/Annotations/labelled/'
for file_name in os.listdir(loc):
    if file_name.endswith(".csv"):
        video_id=file_name[:-4]
        df=pd.read_csv(loc+file_name,names=['sample_number','event'])
        try:
            df[['event','number','scream_type']]=df['event'].str.split('_',expand=True)
            #df['number']=df['number'].astype(int)
            #df
            df['number']=df['number'].astype(int)
            #df
            df1=df.pivot('number','event',['sample_number','scream_type'])
            df1.columns = [ '_'.join(x) for x in df1.columns ]
            try:
                df1=df1[['sample_number_start','sample_number_end','scream_type_start']]
            except:
                df1=df1[['sample_number_start','sample_number_stop','scream_type_start']]
            df1.columns=['sample_start','sample_end','scream_type']
            df1['video_id']=video_id
            df1.to_csv('./resources/dataset/Annotations/processed/'+file_name,header=True, index=False,encoding='utf-8-sig')
        except:
            print(f"The event field has incorrect data for {video_id}")


# Convert all sample numbers to their corresponding timestamps, based on the sample rate from the lookup table so that when resampling the annotations do not get ruined

In [37]:
import pandas as pd
csv_loc = './resources/dataset/Annotations/processed/'
audio_loc = './resources/dataset/Audio/'
output_loc = './resources/dataset/Annotations/timestamps/'

lut = pd.read_csv('./resources/dataset/lookup_new.csv')
for file_name in os.listdir(csv_loc):
    if file_name.endswith(".csv"):
        video_id=file_name[:-4]
        df=pd.read_csv(csv_loc + file_name)
        #tmp = df.join(lut.set_index('video_id'),on = 'video_id')
        tmp=lut[lut['video_id'] == video_id]
        sr = tmp['sample_rate'].iloc[0]
        df.insert(2, 'timestamp_start', round(df['sample_start']/sr,2))
        df.insert(3, 'timestamp_end', round(df['sample_end']/sr,2))
        df.insert(0,'file_name',video_id+'.wav')
        #df.insert()
        #print(sr)
        #print(df)
        df.to_csv(output_loc + video_id + '.csv',header=True, index=False,encoding='utf-8-sig')
            

In [39]:
df.columns

Index(['file_name', 'sample_start', 'sample_end', 'timestamp_start',
       'timestamp_end', 'scream_type', 'video_id'],
      dtype='object')

# Combining all annotations into one file for input into KNN

In [53]:
import pandas as pd
csv_loc = './resources/dataset/Annotations/processed/'
audio_loc = './resources/dataset/Audio/'
timestamp_loc = './resources/dataset/Annotations/timestamps/'

output_loc = './resources/dataset/Annotations/final/'
df1=pd.DataFrame()
i = 0
for file_name in os.listdir(timestamp_loc):
    if file_name.endswith(".csv"):
        #print(file_name)
        if i == 0:
            df1=pd.read_csv(timestamp_loc+file_name,names=['file_name','sample_start','sample_end','timestamp_start','timestamp_end','scream_type','video_id'],header=0)
            i=1
        else:
            temp=pd.read_csv(timestamp_loc+file_name,names=['file_name','sample_start','sample_end','timestamp_start','timestamp_end','scream_type','video_id'],header=0)
            df1=pd.concat([df1,temp],axis=0,ignore_index=True)

df1.to_csv(output_loc + 'annotation.csv',header=True, index=False,encoding='utf-8-sig')

In [52]:
df1

,file_name,sample_start,sample_end,timestamp_start,timestamp_end,scream_type,video_id
0,4600fGWcn9o.wav,1579017,1999183.0,35.81,45.33,midfry,4600fGWcn9o
1,4600fGWcn9o.wav,2020012,2152867.0,45.81,48.82,midfry,4600fGWcn9o
2,4600fGWcn9o.wav,2161887,2486176.0,49.02,56.38,midfry,4600fGWcn9o
3,4600fGWcn9o.wav,2488192,2795520.0,56.42,63.39,clean,4600fGWcn9o
4,4600fGWcn9o.wav,2797600,3030794.0,63.44,68.73,midfry,4600fGWcn9o
...,...,...,...,...,...,...,...
1570,0m5fIHHfJTw.wav,6819456,6915072.0,154.64,156.80,midfry,0m5fIHHfJTw
1571,0m5fIHHfJTw.wav,7056672,7172928.0,160.02,162.65,midfry,0m5fIHHfJTw
1572,0m5fIHHfJTw.wav,7264704,7286880.0,164.73,165.24,midfry,0m5fIHHfJTw
1573,0m5fIHHfJTw.wav,7789376,8205120.0,176.63,186.06,midfry,0m5fIHHfJTw
